link foto :
https://drive.google.com/drive/folders/1ZuGhFurMPqmh62bT0B17qcWONEDRu7UP?usp=sharing

dataset :
https://drive.google.com/file/d/1mYGusV87G5gIjezWvFUKXs-DMBdfC6xz/view?usp=sharing

# Telegram Bot API

In [ ]:
pip install pyTelegramBotAPI

# Load Data and Training Data

In [ ]:
import random
import json
import pickle
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
         word_list = nltk.word_tokenize(pattern)
         words.extend(word_list)
         documents.append((word_list, intent['tag']))
         if intent['tag'] not in classes:
             classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row ])  

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add (Dense(128, input_shape= (len(train_x[0]),), activation='relu'))
model.add (Dropout (0.5))
model.add (Dense(64, activation='relu'))
model.add (Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
hist = model.fit(np.array(train_x), np.array(train_y), epochs=60, batch_size=5, verbose=1)
model.save('chatbotmodel.h5', hist)
print("Done")

#Chatbot Telegram

In [ ]:
import random
import json
import pickle
import numpy as np
from telebot import *
import re
import random

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model



In [ ]:

bot = telebot.TeleBot("5125757787:AAEHjIJ8ug5GaXfZnHDvvOYs_My9AYBydXQ")

#start
@bot.message_handler(commands=['start'])
def send_start(message):
  bot.send_message(message.chat.id,"selamat datang di chatbot kami")

@bot.message_handler(commands=['photo'])
def send_photo(message):
  bot.send_photo(message.chat.id, open('unimed1.jpg', "rb"))
  bot.send_photo(message.chat.id, open('unimed2.jpg', "rb"))

@bot.message_handler(commands=['location'])
def send_location(message):
  bot.send_message(message.chat.id,"https://goo.gl/maps/9kvZ4zps9FLX5FqY9")
  
@bot.message_handler(commands=['help'])
def send_help(message):
  bot.send_message(message.chat.id,"""list menu : 
  /start untuk memulai percakapan dengan chatbot,
  /photo menampilkan foto dari Universitas Negeri Medan, 
  /help memberikan informasi tentang apa saja yang dapat
            dilakukan chatbot.
  /location lokasi dari Universitas Negeri Medan
  untuk informasi lebih detail tentang Universitas Negeri Medan 
  bisa kunjungi website https://www.unimed.ac.id/ 
  """)

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i , r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    
    return result

#chatbot
@bot.message_handler(content_types=['text'])
def chatbot(message):
  teks = message.text
  ints = predict_class(teks)
  res = get_response(ints, intents)
 
  bot.send_message(message.chat.id, res)

bot.infinity_polling()